In [ ]:
#this script will create a graph to be used for the dfs 
#for future, we can develop JSON file saving method instead of using pickle

In [5]:
import pickle
import pandas as pd
pd.set_option('max_colwidth', 200)
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.colors as pltc
import seaborn as sns 
sns.set()
import os
from os import listdir
from os.path import isfile, join
import csv
import numpy as np
%matplotlib inline
from random import sample
from math import sin, cos, sqrt, atan2, radians, acos
import numpy as np
import math
import copy
import time
from datetime import datetime, timedelta

def save_obj(obj, name ):
    with open('obj/'+ name + '.pkl', 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

def load_obj(name ):
    with open('obj/' + name + '.pkl', 'rb') as f:
        return pickle.load(f)


In [6]:
#stop_times table only weekday schedules
#arrival_time_fix and departure_time_fix are the times in seconds 
trip_table_fixed =  pd.read_csv(r'trip_table_fixed.csv', sep=",", engine='python')

In [7]:
########OPTIONAL##########
#select only the 2 hour time frame of 7-9 am from trip_table_fixed
#if the tables are not shrunk, the dfs module will take exponentially longer, and may not finish

trip_table_fixed = trip_table_fixed[trip_table_fixed['arrival_time_fix'].between(25200, 32400, inclusive=True)]



In [8]:
trip_table_fixed.head()

,trip_id,stop_sequence,stop_id,pickup_type,drop_off_type,arrival_time,departure_time,timepoint,stop_headsign,shape_dist_traveled,arrival_time_:,departure_time_:,time_diff,seq,total_time,arrival_time_fix,departure_time_fix,route_short_name,full_bus_name,stop_id_time
385,884545,50,1379,0,0,7:01:00,7:01:00,1,NORTH TFR PT: VIA FORDEM,9.4787,70100,70100,0,0,0,25260,25260,2,2_NORTH TFR PT: VIA FORDEM,1379_25260
386,884545,51,1717,0,0,7:01:41,7:01:41,0,NORTH TFR PT,9.6521,70141,70141,0,0,0,25301,25301,2,2_NORTH TFR PT,1717_25301
387,884545,52,1923,0,0,7:02:32,7:02:32,0,NORTH TFR PT,9.8640,70232,70232,0,0,0,25352,25352,2,2_NORTH TFR PT,1923_25352
388,884545,53,1715,0,0,7:03:19,7:03:19,0,NORTH TFR PT,10.0604,70319,70319,0,0,0,25399,25399,2,2_NORTH TFR PT,1715_25399
389,884545,54,1561,0,0,7:03:58,7:03:58,0,NORTH TFR PT,10.2226,70358,70358,0,0,0,25438,25438,2,2_NORTH TFR PT,1561_25438


In [9]:
#create graph
graph = {}
unique_table=trip_table_fixed.groupby(['route_short_name','trip_id', 'stop_id_time']).size().reset_index()
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'route_short_name']
    graph[bus_name] = {}
for index, row in unique_table.iterrows(): 
    bus_name = unique_table.loc[index, 'route_short_name']
    bus_trip_id = unique_table.loc[index, 'trip_id']
    stop_id = unique_table.loc[index, 'stop_id_time']    
    graph[bus_name][bus_trip_id] = set([])
graph_ready = copy.deepcopy(graph)

In [10]:
graph_ready

{2: {884516: set(),
  884517: set(),
  884518: set(),
  884531: set(),
  884532: set(),
  884533: set(),
  884543: set(),
  884544: set(),
  884545: set(),
  884554: set(),
  884555: set(),
  884556: set(),
  884577: set(),
  884578: set(),
  884579: set(),
  884580: set(),
  884581: set(),
  884605: set(),
  904817: set(),
  904818: set(),
  904819: set(),
  904832: set(),
  904833: set(),
  904834: set(),
  904844: set(),
  904845: set(),
  904846: set(),
  904855: set(),
  904856: set(),
  904857: set(),
  904878: set(),
  904879: set(),
  904880: set(),
  904881: set(),
  904882: set(),
  904906: set()},
 3: {885035: set(),
  885041: set(),
  885045: set(),
  885046: set(),
  885047: set(),
  885064: set(),
  885067: set(),
  885068: set(),
  885083: set(),
  885084: set(),
  885085: set(),
  895174: set(),
  895180: set(),
  895184: set(),
  895185: set(),
  895186: set(),
  895210: set(),
  895211: set(),
  895212: set(),
  895227: set(),
  895228: set(),
  895231: set()},
 4: {8

In [11]:
#create graph form 
#bus#:set([bus_stops])
for index, row in trip_table_fixed.iterrows():
    route_name = row['route_short_name']
    stop_id = row['stop_id_time']
    trip_id = row['trip_id']
    try:
        graph_ready[route_name][trip_id].add(stop_id)
    except:
        print(route_name, stop_id, trip_id)
        break

In [12]:
#since the graph is pre ordered in time, no need to put in any time conditions in the loop!
for key in graph_ready:
    for trip_id in graph_ready[key]:                   
        graph_temp = sorted(list(graph_ready[key][trip_id]), key=lambda x: int(x.split('_')[1]))
        graph_ready[key][trip_id] = graph_temp



In [19]:
#graph_ready

#save_obj(graph_ready, 'graph_ready')
#graph_ready=load_obj('graph_ready')

In [18]:
count = 0 
count_list = []
graph_final = {}
node = trip_table_fixed['stop_id_time'].unique()
for stop_id_time in node:
    stop_id = stop_id_time.split('_')[0]
    stop_time = int(stop_id_time.split('_')[1]) #current time
    #wait time is 30 min 60*30
    stop_time_30 = stop_time+1800
    graph_final[stop_id_time] = [] 
    for bus_name in graph_ready:
        for trip_id in graph_ready[bus_name]:
            stops_list = graph_ready[bus_name][trip_id]
            stops_only_list =[s.split('_', 1)[0] for s in stops_list]
            matching_index = [i for i,d in enumerate(stops_only_list) if d==stop_id]
            #check if the trip_id could possibly have two hits on a stop_id
            if 1 < len(matching_index): 
                for index in matching_index:
                    current_stop = stops_list[index]
                    current_stop_time = int(current_stop.split('_')[1])
                    #check if the stop time for that bus is within 30 min wait
                    if stop_time <= current_stop_time and current_stop_time <= stop_time_30: 
                        #if this trip_id is the last stop, continue to the next trip_id
                        try:
                            next_stop = stops_list[index+1]
                            next_stop_id = next_stop.split('_')[0]
                            next_stop_time = next_stop.split('_')[1]
                        except: 
                            continue
                        travel_time = int(next_stop_time) - int(stop_time)    
                        #find full bus name and put it in
                        row=trip_table_fixed.loc[(trip_table_fixed['trip_id'] == trip_id) & (trip_table_fixed['stop_id_time'] == next_stop)]
                        full = row['full_bus_name'].values[0]
                        dic = {'destination': next_stop, 
                               'travel_time': travel_time, 
                               'trip_id': trip_id, 
                               'bus_name': bus_name, 
                               'full_bus_name': full} 
                        ###final step
                        graph_final[stop_id_time].append(dic)
                    else:
                        continue
                else:
                    continue
            else:      
                try:
                    index = stops_only_list.index(stop_id)
                except:
                    continue
                current_stop = stops_list[index]
                current_stop_time = int(current_stop.split('_')[1])
                #check if the stop time for that bus is within 30 min wait
                if stop_time <= current_stop_time and current_stop_time <= stop_time_30: 
                    #if this trip_id is the last stop, continue to the next trip_id
                    try:
                        next_stop = stops_list[index+1]
                        next_stop_id = next_stop.split('_')[0]
                        next_stop_time = next_stop.split('_')[1]
                    except: 
                        continue
                    travel_time = int(next_stop_time) - int(stop_time)    
                    #find full bus name and put it in
                    row=trip_table_fixed.loc[(trip_table_fixed['trip_id'] == trip_id) & (trip_table_fixed['stop_id_time'] == next_stop)]
                    full = row['full_bus_name'].values[0]
                    dic = {'destination': next_stop, 
                           'travel_time': travel_time, 
                           'trip_id': trip_id, 
                           'bus_name': bus_name, 
                           'full_bus_name': full} 

                    ###final step
                    graph_final[stop_id_time].append(dic)
                else:
                    continue #for trip_id in graph[bus_name]:

In [22]:
#graph_ready
#save_obj(graph_final, 'graph_final')

In [23]:
#graph_final

In [24]:
#this is the graph without any stop destination. Most are because the time limit is until 9 am but some are not.
#must further develop to figure out how to make the graph more accurate
for stop_with_time in graph_final:
    if graph_final[stop_with_time] == []:
            print(stop_with_time)

5100_31860
455_32337
2595_32333
6100_32400
2806_32350
5100_32400
1127_32392
2389_32335
2388_32372
1530_32387
5100_31920
1737_32366
4100_32400
4100_32160
1100_32400
1598_32400
4100_31980
1636_32340
9520_32371
9587_32393
1233_32367
803_32362
2949_32387
926_32386
9810_32377
5100_32160
4100_32100
809_32377
4121_32390
5100_32100
5100_32040
47_28140
47_29940
47_31740
2630_31980
2088_32349
1664_32373
9800_32400
9892_32397
1700_30600
1700_30900
7341_32360
1101_32400
2397_25800
2397_27900
2397_26760
2397_28800
2996_32359
2576_31951
1230_32385
1628_32370
4637_32350
809_32397
8401_32384
1700_32400
2401_32352
1324_32380
6269_32199
2595_32361
4100_31740
7328_32379
7144_32220
167_32347
6350_32400
6476_32040
6120_32367
6667_32100
6476_32400
9800_32220
4812_32380
251_32280
313_31500
941_32220
313_32340
455_32388
2465_32244
2755_32390
2576_31696
2192_32399
2770_32340
568_32400
1704_32394
1233_32398
2477_32374
6516_32340
1660_32367
1700_31380
741_32372
1700_32280
6615_32371
6253_32400
2465_32362
6944_32

In [25]:
#this is one of the stops in the blank.
#it is a drop off but it should lead to the next stop afterwards, but does not. 
#must figure out why some stops are acting this way
trip_table_fixed[trip_table_fixed['stop_id_time']=='2397_27840']

,trip_id,stop_sequence,stop_id,pickup_type,drop_off_type,arrival_time,departure_time,timepoint,stop_headsign,shape_dist_traveled,arrival_time_:,departure_time_:,time_diff,seq,total_time,arrival_time_fix,departure_time_fix,route_short_name,full_bus_name,stop_id_time
127802,901249,82,2397,1,0,7:44:00,7:44:00,1,DROP OFFS ONLY,13.7249,74400,74400,0,0,0,27840,27840,38,38_DROP OFFS ONLY,2397_27840


In [59]:
trip_table_fixed[trip_table_fixed['trip_id']==901249]

,trip_id,stop_sequence,stop_id,pickup_type,drop_off_type,arrival_time,departure_time,timepoint,stop_headsign,shape_dist_traveled,arrival_time_:,departure_time_:,time_diff,seq,total_time,arrival_time_fix,departure_time_fix,route_short_name,full_bus_name,stop_id_time
127751,901249,31,1898,0,0,7:00:10,7:00:10,0,SHEBOYGAN: VIA DENNETT & ATWOOD: VIA WILSON,5.7124,70010,70010,0,0,0,25210,25210,38,38_SHEBOYGAN: VIA DENNETT & ATWOOD: VIA WILSON,1898_25210
127752,901249,32,1682,0,0,7:00:41,7:00:41,0,SHEBOYGAN: VIA DENNETT & ATWOOD: VIA WILSON,5.8157,70041,70041,0,0,0,25241,25241,38,38_SHEBOYGAN: VIA DENNETT & ATWOOD: VIA WILSON,1682_25241
127753,901249,33,1398,0,0,7:01:12,7:01:12,0,SHEBOYGAN: VIA DENNETT & ATWOOD: VIA WILSON,5.9187,70112,70112,0,0,0,25272,25272,38,38_SHEBOYGAN: VIA DENNETT & ATWOOD: VIA WILSON,1398_25272
127754,901249,34,1388,0,0,7:01:39,7:01:39,0,SHEBOYGAN: VIA DENNETT & ATWOOD: VIA WILSON,6.0109,70139,70139,0,0,0,25299,25299,38,38_SHEBOYGAN: VIA DENNETT & ATWOOD: VIA WILSON,1388_25299
127755,901249,35,1784,0,0,7:03:00,7:03:00,1,SHEBOYGAN: VIA WILSON,6.2567,70300,70300,0,0,0,25380,25380,38,38_SHEBOYGAN: VIA WILSON,1784_25380
127756,901249,36,1542,0,0,7:03:41,7:03:41,0,SHEBOYGAN: VIA WILSON,6.3795,70341,70341,0,0,0,25421,25421,38,38_SHEBOYGAN: VIA WILSON,1542_25421
127757,901249,37,1256,0,0,7:04:13,7:04:13,0,SHEBOYGAN: VIA WILSON,6.4747,70413,70413,0,0,0,25453,25453,38,38_SHEBOYGAN: VIA WILSON,1256_25453
127758,901249,38,1447,0,0,7:05:47,7:05:47,0,SHEBOYGAN: VIA WILSON,6.7545,70547,70547,0,0,0,25547,25547,38,38_SHEBOYGAN: VIA WILSON,1447_25547
127759,901249,39,1304,0,0,7:06:29,7:06:29,0,SHEBOYGAN: VIA WILSON,6.8798,70629,70629,0,0,0,25589,25589,38,38_SHEBOYGAN: VIA WILSON,1304_25589
127760,901249,40,1910,0,0,7:07:11,7:07:11,0,SHEBOYGAN: VIA WILSON,7.0058,70711,70711,0,0,0,25631,25631,38,38_SHEBOYGAN: VIA WILSON,1910_25631


In [60]:
trip_table_fixed[trip_table_fixed['stop_id']==2397]

,trip_id,stop_sequence,stop_id,pickup_type,drop_off_type,arrival_time,departure_time,timepoint,stop_headsign,shape_dist_traveled,arrival_time_:,departure_time_:,time_diff,seq,total_time,arrival_time_fix,departure_time_fix,route_short_name,full_bus_name,stop_id_time
40917,891102,82,2397,1,0,7:10:00,7:10:00,1,DROP OFFS ONLY,13.7249,71000,71000,0,0,0,25800,25800,38,38_DROP OFFS ONLY,2397_25800
41059,891080,82,2397,1,0,7:45:00,7:45:00,1,DROP OFFS ONLY,13.7249,74500,74500,0,0,0,27900,27900,38,38_DROP OFFS ONLY,2397_27900
41119,891087,60,2397,1,0,7:26:00,7:26:00,1,DROP OFFS ONLY,9.5523,72600,72600,0,0,0,26760,26760,38,38_DROP OFFS ONLY,2397_26760
41253,891088,60,2397,1,0,8:00:00,8:00:00,1,DROP OFFS ONLY,9.5523,80000,80000,0,0,0,28800,28800,38,38_DROP OFFS ONLY,2397_28800
60873,886621,2,2397,0,0,8:55:05,8:55:05,0,UNION CORNERS: VIA CAMPUS DR: VIA JOHNSON,0.2517,85505,85505,0,0,0,32105,32105,10,10_UNION CORNERS: VIA CAMPUS DR: VIA JOHNSON,2397_32105
127720,901227,82,2397,1,0,7:10:00,7:10:00,1,DROP OFFS ONLY,13.7249,71000,71000,0,0,0,25800,25800,38,38_DROP OFFS ONLY,2397_25800
127802,901249,82,2397,1,0,7:44:00,7:44:00,1,DROP OFFS ONLY,13.7249,74400,74400,0,0,0,27840,27840,38,38_DROP OFFS ONLY,2397_27840
127862,901242,60,2397,1,0,7:25:00,7:25:00,1,DROP OFFS ONLY,9.5523,72500,72500,0,0,0,26700,26700,38,38_DROP OFFS ONLY,2397_26700
127996,901241,60,2397,1,0,7:59:00,7:59:00,1,DROP OFFS ONLY,9.5523,75900,75900,0,0,0,28740,28740,38,38_DROP OFFS ONLY,2397_28740
147657,896798,2,2397,0,0,8:55:05,8:55:05,0,UNION CORNERS: VIA CAMPUS: VIA JOHNSON,0.2517,85505,85505,0,0,0,32105,32105,10,10_UNION CORNERS: VIA CAMPUS: VIA JOHNSON,2397_32105


In [ ]:
#https://www.cityofmadison.com/metro/routes-schedules/route-22

In [22]:
df=trip_table_fixed.loc[(trip_table_fixed['trip_id'] == 904846) & (trip_table_fixed['stop_id_time'] == '1759_25281')]

In [26]:
df['full_bus_name'].values[0]

'2_NORTH TRANSFER'